# Module 3: Linear Regression - Predicting Customer Spend

**Goal:** Build a linear regression model to predict customer total spend from behavioral features.

**Time:** ~45 minutes

**What you'll do:**
1. Load and explore StreamCart customer data
2. Establish a mean baseline
3. Build a simple linear regression (single feature)
4. Build a multiple linear regression (multiple features)
5. Analyze residuals for model diagnostics
6. Interpret coefficients in business terms
7. Communicate results to stakeholders

**Prerequisites:**
- Python basics
- Pandas fundamentals
- Basic statistics (mean, variance)

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

# For reproducibility
np.random.seed(42)

## 1. Load and Explore Data

We'll use StreamCart customer data. Each row is a customer with behavioral features and their total spend.

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

df = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(df):,} customers")
df.head()

In [ ]:
# Let's understand our target variable: total_spend
print("=== Target Variable: total_spend ===")
print(df['total_spend'].describe())

plt.figure(figsize=(10, 4))
plt.hist(df['total_spend'], bins=50, edgecolor='white', alpha=0.7)
plt.xlabel('Total Spend ($)')
plt.ylabel('Count')
plt.title('Distribution of Customer Total Spend')
plt.axvline(df['total_spend'].mean(), color='red', linestyle='--', label=f"Mean: ${df['total_spend'].mean():.0f}")
plt.legend()
plt.show()

In [ ]:
# Explore potential features
feature_cols = ['tenure_months', 'logins_last_30d', 'orders_total', 'avg_order_value', 
                'support_tickets_total', 'logins_per_month_avg']

print("=== Potential Features ===")
for col in feature_cols:
    correlation = df[col].corr(df['total_spend'])
    print(f"{col:25s} correlation with total_spend: {correlation:+.3f}")

## 2. Establish a Baseline

**Always start with a baseline!** The simplest baseline is predicting the mean for everyone.

This tells us: "If our model can't beat predicting the average, it's not learning anything useful."

In [ ]:
# Split data first (important: always split before looking at test data!)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training set: {len(train_df):,} customers")
print(f"Test set: {len(test_df):,} customers")

In [ ]:
# Baseline: predict the training mean for everyone
baseline_prediction = train_df['total_spend'].mean()
print(f"Baseline prediction (mean): ${baseline_prediction:.2f}")

# Evaluate on test set
baseline_mae = mean_absolute_error(test_df['total_spend'], [baseline_prediction] * len(test_df))
baseline_rmse = np.sqrt(mean_squared_error(test_df['total_spend'], [baseline_prediction] * len(test_df)))

print(f"\nBaseline Performance:")
print(f"  MAE:  ${baseline_mae:.2f}")
print(f"  RMSE: ${baseline_rmse:.2f}")

In [ ]:
# ============================================
# SELF-CHECK: Baseline sanity check
# ============================================
assert baseline_mae > 50, "Baseline MAE seems too low - check your data"
assert baseline_mae < 500, "Baseline MAE seems too high - check your data"
print("✓ Baseline looks reasonable")

## 3. Simple Linear Regression (Single Feature)

Let's start with the simplest model: using just **tenure_months** to predict total_spend.

**Hypothesis:** Customers who've been with us longer have spent more.

In [ ]:
# Visualize the relationship first
plt.figure(figsize=(10, 6))
plt.scatter(train_df['tenure_months'], train_df['total_spend'], alpha=0.3, s=20)
plt.xlabel('Tenure (months)')
plt.ylabel('Total Spend ($)')
plt.title('Tenure vs Total Spend')
plt.show()

# What do you observe? Is the relationship roughly linear?

In [ ]:
# Fit simple linear regression
X_train_simple = train_df[['tenure_months']]
X_test_simple = test_df[['tenure_months']]
y_train = train_df['total_spend']
y_test = test_df['total_spend']

model_simple = LinearRegression()
model_simple.fit(X_train_simple, y_train)

# Print the equation
print("=== Simple Linear Regression ===")
print(f"Equation: total_spend = {model_simple.intercept_:.2f} + {model_simple.coef_[0]:.2f} × tenure_months")
print(f"\nInterpretation:")
print(f"  - Intercept (β₀): ${model_simple.intercept_:.2f} = predicted spend for a brand new customer (tenure = 0)")
print(f"  - Coefficient (β₁): ${model_simple.coef_[0]:.2f} = additional spend per month of tenure")

In [ ]:
# Evaluate simple model
y_pred_simple = model_simple.predict(X_test_simple)

simple_mae = mean_absolute_error(y_test, y_pred_simple)
simple_rmse = np.sqrt(mean_squared_error(y_test, y_pred_simple))
simple_r2 = r2_score(y_test, y_pred_simple)

print("=== Simple Model Performance ===")
print(f"  MAE:  ${simple_mae:.2f} (baseline: ${baseline_mae:.2f})")
print(f"  RMSE: ${simple_rmse:.2f} (baseline: ${baseline_rmse:.2f})")
print(f"  R²:   {simple_r2:.3f}")
print(f"\nImprovement over baseline: {(baseline_mae - simple_mae) / baseline_mae * 100:.1f}% reduction in MAE")

In [ ]:
# Visualize the regression line
plt.figure(figsize=(10, 6))
plt.scatter(test_df['tenure_months'], y_test, alpha=0.3, s=20, label='Actual')

# Plot regression line
x_line = np.linspace(0, test_df['tenure_months'].max(), 100)
y_line = model_simple.intercept_ + model_simple.coef_[0] * x_line
plt.plot(x_line, y_line, 'r-', linewidth=2, label='Prediction')

plt.xlabel('Tenure (months)')
plt.ylabel('Total Spend ($)')
plt.title(f'Simple Linear Regression (R² = {simple_r2:.3f})')
plt.legend()
plt.show()

In [ ]:
# ============================================
# SELF-CHECK: Simple model should beat baseline
# ============================================
assert simple_mae < baseline_mae, "Model should beat the baseline!"
assert model_simple.coef_[0] > 0, "Tenure coefficient should be positive (longer tenure = more spend)"
print("✓ Simple model passes sanity checks")

## 4. Multiple Linear Regression

Now let's add more features. Multiple regression uses the equation:

**y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ**

Each coefficient tells us: "Holding all other features constant, a 1-unit increase in this feature is associated with βᵢ change in the target."

In [ ]:
# Select features for multiple regression
features = ['tenure_months', 'logins_last_30d', 'orders_total', 'avg_order_value']

X_train_multi = train_df[features].fillna(0)
X_test_multi = test_df[features].fillna(0)

model_multi = LinearRegression()
model_multi.fit(X_train_multi, y_train)

# Print coefficients
print("=== Multiple Linear Regression ===")
print(f"Intercept: ${model_multi.intercept_:.2f}\n")
print("Coefficients:")
for feature, coef in zip(features, model_multi.coef_):
    print(f"  {feature:20s}: {coef:+.2f}")

In [ ]:
# Evaluate multiple regression model
y_pred_multi = model_multi.predict(X_test_multi)

multi_mae = mean_absolute_error(y_test, y_pred_multi)
multi_rmse = np.sqrt(mean_squared_error(y_test, y_pred_multi))
multi_r2 = r2_score(y_test, y_pred_multi)

print("=== Multiple Model Performance ===")
print(f"  MAE:  ${multi_mae:.2f}")
print(f"  RMSE: ${multi_rmse:.2f}")
print(f"  R²:   {multi_r2:.3f}")

print(f"\n=== Comparison ===")
print(f"  Baseline MAE:      ${baseline_mae:.2f}")
print(f"  Simple Model MAE:  ${simple_mae:.2f}")
print(f"  Multiple Model MAE: ${multi_mae:.2f}")

In [ ]:
# ============================================
# SELF-CHECK: Multiple model should improve R²
# ============================================
assert multi_r2 > simple_r2, "Adding features should improve R² (on training data at least)"
print(f"✓ R² improved from {simple_r2:.3f} to {multi_r2:.3f}")

## 5. Residual Analysis

Residuals = Actual - Predicted

A good model has residuals that are:
- **Random** (no patterns)
- **Centered around zero**
- **Roughly constant spread** (homoscedasticity)

Patterns in residuals tell us the model is systematically wrong.

In [ ]:
# Calculate residuals
residuals = y_test - y_pred_multi

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Residuals vs Predicted
axes[0].scatter(y_pred_multi, residuals, alpha=0.3, s=20)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted Value ($)')
axes[0].set_ylabel('Residual ($)')
axes[0].set_title('Residuals vs Predicted')

# Plot 2: Histogram of residuals
axes[1].hist(residuals, bins=50, edgecolor='white', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_xlabel('Residual ($)')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of Residuals')

# Plot 3: Actual vs Predicted
axes[2].scatter(y_test, y_pred_multi, alpha=0.3, s=20)
max_val = max(y_test.max(), y_pred_multi.max())
axes[2].plot([0, max_val], [0, max_val], 'r--', label='Perfect prediction')
axes[2].set_xlabel('Actual Total Spend ($)')
axes[2].set_ylabel('Predicted Total Spend ($)')
axes[2].set_title('Actual vs Predicted')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\nWhat to look for:")
print("  - Left plot: Should show random scatter around zero (no patterns)")
print("  - Middle plot: Should be roughly bell-shaped, centered at zero")
print("  - Right plot: Points should cluster around the diagonal line")

## 6. Coefficient Interpretation Workshop

This is where we translate model outputs into business insights.

In [ ]:
# Create a coefficient summary table
coef_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': model_multi.coef_,
    'Unit': ['months', 'logins', 'orders', 'dollars']
})
coef_df['Interpretation'] = [
    f"Each additional month of tenure → ${coef_df.loc[0, 'Coefficient']:.2f} more spend",
    f"Each additional login (last 30d) → ${coef_df.loc[1, 'Coefficient']:.2f} more spend",
    f"Each additional order → ${coef_df.loc[2, 'Coefficient']:.2f} more spend",
    f"Each $1 higher avg order value → ${coef_df.loc[3, 'Coefficient']:.2f} more total spend"
]

print("=== Coefficient Interpretations ===")
print(coef_df.to_string(index=False))

In [ ]:
# Visualize coefficient magnitudes
# Note: These are NOT directly comparable due to different scales!

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw coefficients
colors = ['green' if c > 0 else 'red' for c in model_multi.coef_]
axes[0].barh(features, model_multi.coef_, color=colors, alpha=0.7)
axes[0].axvline(x=0, color='black', linewidth=0.5)
axes[0].set_xlabel('Coefficient Value')
axes[0].set_title('Raw Coefficients (NOT directly comparable!)')

# Standardized coefficients (for fair comparison)
X_train_std = (X_train_multi - X_train_multi.mean()) / X_train_multi.std()
model_std = LinearRegression()
model_std.fit(X_train_std, y_train)

colors_std = ['green' if c > 0 else 'red' for c in model_std.coef_]
axes[1].barh(features, model_std.coef_, color=colors_std, alpha=0.7)
axes[1].axvline(x=0, color='black', linewidth=0.5)
axes[1].set_xlabel('Standardized Coefficient')
axes[1].set_title('Standardized Coefficients (comparable importance)')

plt.tight_layout()
plt.show()

print("\nKey insight: Raw coefficients depend on feature scales.")
print("To compare feature importance, use standardized coefficients or feature importance scores.")

In [ ]:
# TODO: Write a business interpretation for one coefficient
#
# Fill in the blank:
# "Holding all else equal, one more ______ in the last 30 days is associated with $______ more total spend."
#
# Your answer:
your_interpretation = """
Holding all else equal, one more ______ is associated with $______ more total spend.
"""
print(your_interpretation)

## 7. Explain It to a PM

The final step: translate your model into a message a product manager can act on.

In [ ]:
# Summary statistics for your update
print("=== Summary for PM Update ===")
print(f"\nModel Performance:")
print(f"  - R² = {multi_r2:.3f} ({multi_r2*100:.1f}% of spend variation explained)")
print(f"  - Average prediction error: ${multi_mae:.2f}")
print(f"  - Improvement over baseline: {(baseline_mae - multi_mae) / baseline_mae * 100:.1f}%")

print(f"\nKey Drivers of Customer Spend:")
for feature, coef in sorted(zip(features, model_std.coef_), key=lambda x: abs(x[1]), reverse=True):
    direction = "↑" if coef > 0 else "↓"
    print(f"  {direction} {feature}: {abs(coef):.2f} (standardized importance)")

In [ ]:
# TODO: Write your PM update (150-250 words)
#
# Include:
# 1. What the model predicts
# 2. How accurate it is (in plain language)
# 3. Key drivers discovered
# 4. One actionable recommendation
#
# Avoid: jargon like "R²", "coefficients", "residuals"

pm_update = """
## Customer Spend Prediction Model - Summary

[YOUR UPDATE HERE]

**What we built:**


**How well it works:**


**Key findings:**


**Recommendation:**


"""

print(pm_update)

---

## Summary

In this lab, you learned to:

1. **Always establish a baseline** - Predicting the mean is the simplest model
2. **Build simple models first** - Single feature before multiple features
3. **Interpret coefficients** - Each one has a business meaning (with units!)
4. **Check residuals** - Patterns reveal model problems
5. **Communicate to stakeholders** - Translate metrics into decisions

### Key Metrics Learned

| Metric | Meaning | Business Translation |
|--------|---------|---------------------|
| MAE | Average absolute error | "Our predictions are off by $X on average" |
| RMSE | Root mean squared error | "Typical error, but big misses hurt more" |
| R² | Variance explained | "Our model explains X% of why customers differ in spend" |

### Next Steps

- Try the **Debug Drill** to find bugs in regression models
- Review the **Cheatsheet** for quick reference
- Take the **Quiz** to test your understanding

In [ ]:
# ============================================
# FINAL SELF-CHECK: Did you complete everything?
# ============================================
print("Lab Completion Checklist:")
print(f"  [✓] Loaded data: {len(df):,} customers")
print(f"  [✓] Baseline MAE: ${baseline_mae:.2f}")
print(f"  [✓] Simple model R²: {simple_r2:.3f}")
print(f"  [✓] Multiple model R²: {multi_r2:.3f}")
print(f"  [✓] Model beats baseline: {multi_mae < baseline_mae}")
print(f"  [ ] TODO: Coefficient interpretation filled in")
print(f"  [ ] TODO: PM update written")

print("\n🎉 Great work! Make sure to complete the TODO sections.")